In [5]:
import cv2
import numpy as np
import os
from pathlib import Path

# ======================
# 0) Dosya adı
# ======================
IMAGE_PATH = "fis.png"
assert os.path.exists(IMAGE_PATH), f"{IMAGE_PATH} yok. Bu klasörde şunlar var: {os.listdir()}"

# ======================
# 1) output klasörü
# ======================
OUT = Path("output_4")
OUT.mkdir(exist_ok=True)

def kaydet(ad, img):
    cv2.imwrite(str(OUT / ad), img)

# ======================
# 2) resmi oku
# ======================
img = cv2.imread(IMAGE_PATH)
assert img is not None, "Resim okunamadı."
H, W = img.shape[:2]
kaydet("01_orjinal.png", img)

# ==================================================
# 3) FİŞİ BUL (HSV ile açık renk kağıdı yakala)
# ==================================================
hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

# Kağıt: düşük doygunluk + yüksek parlaklık
lower = np.array([0, 0, 140])
upper = np.array([179, 90, 255])
mask = cv2.inRange(hsv, lower, upper)

# Maske temizleme
k = cv2.getStructuringElement(cv2.MORPH_RECT, (9, 9))
mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, k, iterations=2)
mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN,  k, iterations=1)

cnts, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
cnts = sorted(cnts, key=cv2.contourArea, reverse=True)

fis_box = None
for c in cnts:
    x, y, w, h = cv2.boundingRect(c)
    oran = (w * h) / (W * H)
    if 0.10 < oran < 0.95:
        fis_box = (x, y, w, h)
        break

assert fis_box is not None, "Fiş bulunamadı (ışık/kontrast düşük olabilir)."

x, y, w, h = fis_box

img_fis_yesil = img.copy()
cv2.rectangle(img_fis_yesil, (x, y), (x+w, y+h), (0, 255, 0), 3)
kaydet("02_fis_yesil_kutu.png", img_fis_yesil)

# Fiş crop
fis = img[y:y+h, x:x+w].copy()

# ==================================================
# 4) BARKOD BUL ve YEŞİL İŞARETLE
# ==================================================
barcode_detector = cv2.barcode_BarcodeDetector()
ok, boxes = barcode_detector.detect(fis)

img_barkod_yesil = img.copy()
barcode_mask = np.zeros((h, w), dtype=np.uint8)  # fiş içinde barkod alanı = 255

if ok:
    for box in boxes:
        pts = box.astype(int)

        # Global koordinat
        pts_g = pts.copy()
        pts_g[:, 0] += x
        pts_g[:, 1] += y

        cv2.polylines(img_barkod_yesil, [pts_g], True, (0, 255, 0), 3)
        cv2.fillPoly(barcode_mask, [pts], 255)

kaydet("03_barkod_yesil_kutu.png", img_barkod_yesil)

# ==================================================
# 5) YAZILARI BLURLA (BLACKHAT ile maske)
#    Barkod alanına blur uygulanmaz.
# ==================================================
gray = cv2.cvtColor(fis, cv2.COLOR_BGR2GRAY)

bh_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (15, 15))
blackhat = cv2.morphologyEx(gray, cv2.MORPH_BLACKHAT, bh_kernel)
blackhat = cv2.normalize(blackhat, None, 0, 255, cv2.NORM_MINMAX)

_, text_mask = cv2.threshold(blackhat, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

text_mask = cv2.morphologyEx(
    text_mask,
    cv2.MORPH_CLOSE,
    cv2.getStructuringElement(cv2.MORPH_RECT, (9, 3)),
    iterations=1
)

text_mask = cv2.morphologyEx(
    text_mask,
    cv2.MORPH_OPEN,
    cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3)),
    iterations=1
)

# barkodu maske dışına at
text_mask[barcode_mask == 255] = 0

# blur uygula
blurred = cv2.GaussianBlur(fis, (21, 21), 0)
fis_blur = fis.copy()
fis_blur[text_mask == 255] = blurred[text_mask == 255]

img_yazi_blur = img.copy()
img_yazi_blur[y:y+h, x:x+w] = fis_blur
kaydet("04_yazi_blur.png", img_yazi_blur)

print("BİTTİ -> output_4 klasörüne bak (4 çıktı):")
print("01_orjinal.png")
print("02_fis_yesil_kutu.png")
print("03_barkod_yesil_kutu.png")
print("04_yazi_blur.png")


BİTTİ -> output_4 klasörüne bak (4 çıktı):
01_orjinal.png
02_fis_yesil_kutu.png
03_barkod_yesil_kutu.png
04_yazi_blur.png
